In [1]:
#| default_exp trigonometry

In [3]:
#| export

import jax
import jax.numpy as jnp

import functools

In [4]:
#| export

from jaxtyping import Float

In [5]:

#| hide

jax.config.update("jax_enable_x64", True)
jax.config.update("jax_debug_nans", False)

In [6]:
#| hide

import jaxtyping


In [7]:

#| hide

%load_ext jaxtyping
%jaxtyping.typechecker beartype.beartype

## Basic trigonometry

For example, the circumcenter of a triangle, which is the position of the dual Voronoi vertex.

### Coding style notes

Throughout, we will (attempt to) provide a type signature for all functions. To do so for array-based functions, we use  [jaxtyping](https://docs.kidger.site/jaxtyping).

### JAX

The aim is to create a triangulation datastructure compatible with the JAX library for automatic differentiation and numerical computing. In practice, this means that we use `jnp` (=`jax.numpy`) instead of `numpy`, and make sure our code follows JAX's functional programming paradigm (see [JAX- the sharp bits](https://docs.jax.dev/en/latest/notebooks/Common_Gotchas_in_JAX.html)). There is also some extra legwork to register any new classes with JAX.

In [8]:
#| export

## trig functions - we can use vmap to vectorize them

@functools.partial(jax.jit, static_argnames=['zero_clip'])
def get_circumcenter(a: Float[jax.Array, " dim"],
                     b: Float[jax.Array, " dim"],
                     c: Float[jax.Array, " dim"], zero_clip: float = 1e-10) -> Float[jax.Array, " dim"]:
    """Return circumcenter coordinates of triangle with vertices a, b, c"""    
    # compute using barycentric coordinates. Start by computing the edge lengths:
    la, lb, lc = (jnp.linalg.norm(b-c), jnp.linalg.norm(c-a), jnp.linalg.norm(a-b,))
    ba, bb, bc = (la**2*(lb**2+lc**2-la**2), lb**2*(lc**2+la**2-lb**2), lc**2*(la**2+lb**2-lc**2))
    u = (ba*a+bb*b+bc*c)/jnp.clip(ba+bb+bc, zero_clip)  # avoid div by zero for degenerate triangles
    return u

def get_triangle_area(a: Float[jax.Array, " dim"],
                      b: Float[jax.Array, " dim"],
                      c: Float[jax.Array, " dim"]) -> Float[jax.Array, ""]:
    """Area of triangle with vertices a, b, c. Works in any dimension >=2."""
    return 0.5 * jnp.linalg.norm(jnp.cross(b - a, c - a))
    
def get_polygon_area(pts: Float[jax.Array, "n_vertices 2"]) -> Float[jax.Array, ""]:
    """Area of 2D polygon assuming no self-intersection."""
    return jnp.sum(pts[:,0]*jnp.roll(pts[:,1], 1) - jnp.roll(pts[:,0], 1)*pts[:,1])/2

@functools.partial(jax.jit, static_argnames=['zero_clip'])
def get_voronoi_corner_area(a: Float[jax.Array, "2"],
                            b: Float[jax.Array, "2"],
                            c: Float[jax.Array, "2"], zero_clip: float=1e-10) -> Float[jax.Array, ""]:
    """
    Compute Voronoi area at corner a of triangle abc. Returns zero for a degenerate triangle. 2d only atm.
    
    TO DO: does this yield _correct_ results for self-intersecting corner slices (circumcenter outside of triangle)?
    """
    u = get_circumcenter(a, b, c)
    a_corner = get_polygon_area(jnp.stack([a, (a-b)/2, u, (a-c)/2], axis=0)) # Voronoi edges are midpoints of triangle edges
    a_triangle = get_polygon_area(jnp.stack([a, b, c], axis=0))
    return jnp.where(jnp.abs(a_triangle) > zero_clip, a_corner, 0.0)

@functools.partial(jax.jit, static_argnames=['zero_clip'])
def get_voronoi_corner_perimeter(a: Float[jax.Array, "2"],
                                 b: Float[jax.Array, "2"],
                                 c: Float[jax.Array, "2"], zero_clip: float=1e-10) -> Float[jax.Array, ""]:
    """Compute contribution to Voronoi perimeter at corner a of triangle abc. Can be negative! 2d only atm."""
    u = get_circumcenter(a, b, c)
    e1 = u - (a + b) / 2    # Voronoi edges are midpoints of triangle edges
    e2 = u - (a + c) / 2
    n1 = (b-a)/jnp.clip(jnp.linalg.norm(b-a), zero_clip)
    n2 = (a-c)/jnp.clip(jnp.linalg.norm(a-c), zero_clip)
    return jnp.cross(n1, e1) + jnp.cross(n2, e2)
    
def get_angle_between_vectors(a: Float[jax.Array, " dim"],
                              b: Float[jax.Array, " dim"]) -> Float[jax.Array, ""]:
    """Angle between two vectors"""
    inner = a.dot(b) / (jnp.linalg.norm(a) * jnp.linalg.norm(b))
    return jnp.arccos(jnp.clip(inner, -1, 1))

def get_rot_mat(theta: float) -> Float[jax.Array, "2 2"]:
    """Get rotation matrix from angle in radians."""
    return jnp.array([[jnp.cos(theta), jnp.sin(theta)],[-jnp.sin(theta), jnp.cos(theta)]])

In [9]:
get_voronoi_corner_perimeter(jnp.array([0., 0.]), jnp.array([1., 0.]), jnp.array([0., 1.]))

Array(1., dtype=float64)

In [10]:
get_polygon_area(jnp.array([[0.,0.], [0.,1.], [1.,0.]]) ), get_triangle_area(*jnp.array([[0.,0.], [0.,1.], [1.,0.]]) )

(Array(0.5, dtype=float64), Array(0.5, dtype=float64))

In [11]:
get_voronoi_corner_area(jnp.array([0.,0.]), jnp.array([0.,1.]),  jnp.array([1.,0.]))

Array(-0.25, dtype=float64)

In [12]:
get_circumcenter(jnp.array([0.,0.]), jnp.array([0.,1.]),  jnp.array([1.,0.]))

Array([0.5, 0.5], dtype=float64)

In [13]:
get_circumcenter(jnp.array([1.,0.]), jnp.array([1.,0.]),  jnp.array([0.,1.]))

Array([0., 0.], dtype=float64)

In [14]:
jnp.cross(jnp.array([1,0]), jnp.array([0, 1]) )

Array(1, dtype=int64)

In [15]:
jnp.cross(jnp.array([1,0,0]), jnp.array([0, 1, 0]) )

Array([0, 0, 1], dtype=int64)